# Predicting Credit Card Application Approvals

<br />
Commercial banks receive myriads of credit card applications. Manually analyzing these applications is mundane, error-prone, and time-consuming, as many factors are considered: high loan balances, low income levels, too many inquiries on a credit card, etc. In this project, I will build an automatic credit card approval predictor using machine learning. 

We will use the Credit Card Approval dataset from the UCI ML Repository, which contains confidential data, and anonymized feature names. First, let's load and look at the dataset. 

In [1]:
import pandas as pd

# load dataset
cc_apps = pd.read_csv('datasets/cc_approvals.data', header=None)

# inspect credit card apps
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
